In [16]:
import kagglegym

# Create environment
env = kagglegym.make()
# Get first observation
observation = env.reset()

observation.features.shape

(968, 110)

In [17]:
# hyperparameters
batch_size = 5 # every how many episodes to do a param update?
learning_rate = 1e-2 # feel free to play with this to train faster or more stably.
gamma = 0.99 # discount factor for reward

D = observation.features.head().shape[1] # input dimensionality
H = int(D / 3) # number of hidden layer neurons

[D, H]

[110, 36]

In [81]:
import tensorflow as tf

tf.reset_default_graph()

#This defines the network as it goes from taking an observation of the environment to 
#giving a probability of chosing to the action of moving left or right.
observations = tf.placeholder(tf.float32, [None,D] , name="input_x")
W1 = tf.get_variable("W1", shape=[D, H],
           initializer=tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(observations,W1))
W2 = tf.get_variable("W2", shape=[H, 1],
           initializer=tf.contrib.layers.xavier_initializer())
score = tf.matmul(layer1,W2)
probability = tf.nn.sigmoid(score)

#From here we define the parts of the network needed for learning a good policy.
tvars = tf.trainable_variables()
input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.placeholder(tf.float32,name="reward_signal")

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
#TODO fix loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loglik = (input_y - probability)
loss = -tf.reduce_mean(loglik * advantages) 
newGrads = tf.gradients(loss,tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradeients after every episode in order to account for noise in the reward signal.
adam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
# Placeholders to send the final gradients through when we update.
W1Grad = tf.placeholder(tf.float32,name="batch_grad1") 
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
#
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))


In [112]:
import numpy as np

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    # in Python 2:
    # for t in reversed(xrange(0, r.size)):
    for t in reversed(range(0, r.shape[0])):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r


In [113]:
class Debug:
    def __init__(self, state):
        self.state = state

    def log(self, msg):
        if self.state % 1 == 0:
            print(msg)


In [114]:
# Here's an example of loading the CSV using Pandas's built-in HDF5 support:
import pandas as pd

with pd.HDFStore("../input/train.h5", "r") as train:
    # Note that the "train" dataframe is the only dataframe in the file
    df = train.get("train")

df.head(3)

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
0,10,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.335633,0.113292,1.621238,...,0.775208,NaN,NaN,NaN,-0.414776,NaN,NaN,-2.0,NaN,-0.011753
1,11,0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,0.004413,0.114285,-0.210185,...,0.025590,NaN,NaN,NaN,-0.273607,NaN,NaN,-2.0,NaN,-0.001240
2,12,0,-0.010622,-0.050577,3.379575,-0.157525,-0.068550,-0.155937,1.219439,-0.764516,...,0.151881,NaN,NaN,NaN,-0.175710,NaN,NaN,-2.0,NaN,-0.020940


In [132]:
# Create environment
env = kagglegym.make()
# Get first observation
observation = env.reset()

init = tf.global_variables_initializer()


def extend(x):
    if x.shape[0] > 967:
        return x
    keys = list(x.keys())
    zeros = [.0] * len(keys)
    tail = pd.DataFrame(dict(zip(keys, zeros)), index=[967])
    x = pd.concat([x, tail])
    return x

class State:
    def __init__(self, observation, reward = .0, done = False, info = {}, y = []):
        self.observation = observation
        self.action = observation.target
        self.reward = reward
        self.done = done
        self.info = info
        self.y = y
    
    @property
    def y(self):
        return self.action["y"];

    @y.setter
    def y(self, y):
        rows = self.action.shape[0]
        self.action["y"] = y if len(y) == rows else [.0] * rows
        self.action = extend(self.action)        

class Agent:
    def __init__(self, sess, env):
        self.sess = sess
        self.env = env
    
    def step(self, state, debug = Debug(-1)):
        observation = state.observation
        x = observation.features
        x = extend(x)
        # Run the policy network and get an action to take. 
        y = self.sess.run(probability,feed_dict={observations: x})
        debug.log("len(Y)={}".format(len(y)))
        state.y = y
        # step the environment and get new measurements
        if observation.target.shape[0] == state.y.shape[0]:
            observation, reward, done, info = self.env.step(state.action)
        else:
            observation, reward, done, info = self.env.step(observation.target)
        debug.log("observation.shape={}".format(observation.target.shape))
        return State(observation, reward, done, info, y)
        
        
timestamp = -1
# Launch the graph
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    print("gradBuffer", len(gradBuffer), len(gradBuffer[0]), len(gradBuffer[1]))
    
    rewards = []
    done = False
    epy = []
    agent = Agent(sess, env)
    state = State(observation, reward)
    while not done:
        x = state.observation.features.head(968)
        x = x.fillna(.0)
        x = extend(x)
        # log
        t = x["timestamp"][0]
        if timestamp == t or t == 910 or t == 911:
            print("Skip repeated timestamp {}".format(timestamp))
            debug.log(action.tail(3))
            # step the environment and get new measurements
            state = agent.step(state)
            continue
            
        timestamp = x["timestamp"][0]
        #timestamp = 0
        debug = Debug(timestamp)
        debug.log(x.tail(3))
        debug.log("\Timestamp #{}".format(timestamp))

        debug.log("Action is {0} {1}".format(state.action.shape, state.action.tail(3)))
        state = agent.step(state)
        # log
        debug.log(state.action.tail(3))
        debug.log("Reward is {}".format(state.reward))
        
        # record reward (has to be done after we call step() to get reward for previous action)
        perfect_action = df[df["timestamp"] == state.observation.features["timestamp"][0]][["id", "y"]] \
            .reset_index(drop=True)
        pa = perfect_action["y"].head(968)
        pa = pa.fillna(.0)
        pa = extend(pa)
        # extend to 968
        #for i in range(pa.shape[0]+1, 969):
        #    pa = pa.append(pd.Series(.0, index=[i]))
        #pa = np.nan_to_num(perfect_action["y"])
                
        y = state.action["y"]
        debug.log("Perfect action {}".format(pa.tail(3)))
        rewards = pa - y.fillna(.0)
        rewards = -np.abs(rewards) #* 10
        rewards = np.nan_to_num(rewards)
        debug.log("Rewards .shape {}".format(len(rewards)))
        #rewards = np.vstack(rewards)
        
        if len(epy) == 0 or y.equals(epy): # protect gradients
            print("Set epy to perfect action")
            epy = pa
            
        epy = np.vstack(epy)
        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(rewards)
        # size the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)
        # Get the gradient for this episode, and save it in the gradBuffer
        tGrad = sess.run(newGrads,feed_dict={observations: x, input_y: epy, advantages: discounted_epr})
        for ix,grad in enumerate(tGrad):
            #debug.log("grad-{0} is {1}".format(str(ix), grad))
            gradBuffer[ix] += grad
        
        # If we have completed enough episodes, then update the policy network with our gradients.
        sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
        for ix,grad in enumerate(gradBuffer):
            gradBuffer[ix] = grad * 0

        epy = y
    
        if timestamp > 910:
            break

gradBuffer 2 110 36
       id  timestamp  derived_0  derived_1  derived_2  derived_3  derived_4  \
965  2154        907   0.073150  -0.015944   0.292708   0.335373    0.04207   
966  2155        907   0.019957  -0.053259   0.000000  -0.010258    0.00000   
967  2156        907  -0.011181  -0.047295   0.000000   0.079977    0.00000   

     fundamental_0  fundamental_1  fundamental_2      ...       technical_35  \
965      -0.228946       0.176791      -0.157165      ...          -0.057939   
966      -0.095227      -0.095520       0.000000      ...           0.157331   
967       0.303686       0.230367       0.000000      ...          -0.221998   

     technical_36  technical_37  technical_38  technical_39  technical_40  \
965      0.116224 -7.629580e-01 -2.924564e-15 -7.629580e-01      0.073505   
966      0.018996 -6.040228e-04 -6.040228e-04 -6.040228e-04     -0.073433   
967     -0.304919 -1.677802e-16 -1.677802e-16 -1.677802e-16      0.069225   

     technical_41  technical_42  

InvalidArgumentError: Incompatible shapes: [967,1] vs. [968,1]
	 [[Node: sub = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_input_y_0, Sigmoid)]]

Caused by op 'sub', defined at:
  File "/opt/conda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/opt/conda/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/opt/conda/lib/python3.5/site-packages/tornado/ioloop.py", line 887, in start
    handle

In [39]:
env = kagglegym.make()
observation = env.reset()

while True:
    target = observation.target
    timestamp = observation.features["timestamp"][0]
    if timestamp % 100 == 0:
        print("Timestamp #{}".format(timestamp))

    observation, reward, done, info = env.step(target)
    if done:        
        break
        
info

Timestamp #1000
Timestamp #1100
Timestamp #1200
Timestamp #1300
Timestamp #1400
Timestamp #1500
Timestamp #1600
Timestamp #1700
Timestamp #1800


{'public_score': -0.0083462397338969572}

In [129]:
import tensorflow.contrib.slim as slim

class agent():
    def __init__(self, lr, s_size,a_size,h_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None,s_size],dtype=tf.float32)
        hidden = slim.fully_connected(self.state_in,h_size,biases_initializer=None,activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden,a_size,activation_fn=tf.nn.softmax,biases_initializer=None)
        self.chosen_action = tf.argmax(self.output,1)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        
        self.indexes = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)

        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss,tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))


In [130]:
# TODO implement arbitrary actions from https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb

tf.reset_default_graph() #Clear the Tensorflow graph.
myAgent = agent(lr=1e-2,s_size=4,a_size=2,h_size=8) #Load the agent.

init = tf.global_variables_initializer()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

/opt/conda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
